<a href="https://colab.research.google.com/github/datasistah/ml_sytem_design_course/blob/main/Falcon_PEFT_6_22_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U git+https://github.com/lvwerra/trl.git git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/accelerate.git git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s et

#### Imports

In [ ]:
import torch, einops
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft.tuners.lora import LoraLayer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

#### Load Quantized Model



In [ ]:
model_id = "ybelkada/falcon-7b-sharded-bf16"
compute_dtype = getattr(torch, "float16")

#special config for bitsandbytes that enables 4bit quantization
bnb_config = BitsAndBytesConfig(
              load_in_4bit=True,
              bnb_4bit_quant_type="nf4",
              bnb_4bit_compute_dtype=compute_dtype,
              bnb_4bit_use_double_quant=True,
              )

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
                            model_id,
                            quantization_config=bnb_config,
                            trust_remote_code=True,
                            )


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

#### Prepare Model for Training


*   activate gradient_checkpointing which significally reduces memory
*   print trainable parameters




In [ ]:
from peft import prepare_model_for_kbit_training


model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

#### Create LoRa Config and Appply to Model


*   This is when we add the low ranking adapters/parameters to the model which are the only parameters that will be trained.

*   Increasing the "r" parameter in LoraConfig will add more adapters to the model.



In [ ]:
 from peft import LoraConfig, get_peft_model

 peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "query_key_value"
        ]
    )

 model = get_peft_model(model, peft_config)
 print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


#### Prepare Data

In [ ]:
# https://huggingface.co/datasets/timdettmers/openassistant-guanaco

from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"

data= load_dataset(dataset_name, split='train')
tokenizer.pad_token = tokenizer.eos_token

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/timdettmers___json/timdettmers--openassistant-guanaco-6126c710748182cf/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


#### Example Before Fine Tuning

*   using data without training
*   convert text to pytorch tensors



In [ ]:
text = '''### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.'''
encoding = tokenizer(text,return_tensors="pt").to("cuda:0")


In [ ]:
encoding

{'input_ids': tensor([[19468,  6823,    37,  1853,   299,  3131,   241,  1866,  9705,   544,
           248, 23335,   275,   248,  1650,   204,    13,  3032,   387,  1799,
           100,    13,   272, 17750,    42,  4012,   745,  6432,  3236,   271,
          2879,   988,   387,  1799,   424,   272,   248, 17170,  1208,   273,
         27421,  5228,  1959,    25]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [ ]:
output = model.generate(input_ids=encoding.input_ids,
                        attention_mask=encoding.attention_mask,
                        eos_token_id=tokenizer.eos_token_id,
                        max_new_tokens=256,
                        top_k=0,
                        do_sample=False)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [ ]:
#output

In [ ]:
print(tokenizer.decode(output[0], skip_special_tokens=True))

### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.

Monopsony is a market structure where there is only one buyer. This is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in t

In [ ]:
pwd

'/content'

In [ ]:
output_dir = "/drive/MyDrive/20230622_outputs/"

#### Training


*   TRL- HuggingFace "Transformer Reinforcement Learning" library
*   SFTTrainer- "supervised fine-tuning"
*   "paged_adamw_32bit" is powerful optimizer, helps to avoid memory overload



In [ ]:
data

Dataset({
    features: ['text'],
    num_rows: 9846
})

In [ ]:
#import supervised fine-tuning from HF library
#setup the trainer

from trl import SFTTrainer

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=30,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=500,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=128,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=True,
)
model.config.use_cache = False  # silence the warnings
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


Step,Training Loss
1,1.789000
2,2.063300
3,1.823700
4,1.716000
5,1.909000
6,1.530800
7,1.791000
8,1.799800
9,1.700800
10,1.905100


TrainOutput(global_step=500, training_loss=1.698545113325119, metrics={'train_runtime': 3565.3022, 'train_samples_per_second': 2.244, 'train_steps_per_second': 0.14, 'total_flos': 2.0385757396992e+16, 'train_loss': 1.698545113325119, 'epoch': 0.81})

#### Save Model


*   local directory
*   HuggingFace



In [ ]:
#Save to directory
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)

In [ ]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [ ]:
#push model to HuggingFace Hub
model_to_save.push_to_hub(repo_id='datasistah/qlora_falcon_20230622')

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasistah/qlora_falcon_20230622/commit/ed46282fbf4a72aeb15a0e49cd14e081fd8032c8', commit_message='Upload model', commit_description='', oid='ed46282fbf4a72aeb15a0e49cd14e081fd8032c8', pr_url=None, pr_revision=None, pr_num=None)

#### Import Model for Inference

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("datasistah/qlora_falcon_20230622")
model = AutoModelForCausalLM.from_pretrained("ybelkada/falcon-7b-sharded-bf16", trust_remote_code=True)
model = PeftModel.from_pretrained(model, "datasistah/qlora_falcon_20230622")


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)

In [ ]:
text2 = '''### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.'''
encoding = tokenizer(text2,return_tensors="pt").to("cuda:0")





In [ ]:
output2 = model.generate(input_ids=encoding.input_ids,
                        attention_mask=encoding.attention_mask,
                        eos_token_id=tokenizer.eos_token_id,
                        max_new_tokens=256,
                        top_k=0,
                        do_sample=False)



In [ ]:
print(tokenizer.decode(output2[0], skip_special_tokens=True))

### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.

Monopsony is a market structure where there is only one buyer. This is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in the real world. Monopsony is a market structure that is not very common in t

#### Evaluation

*   Rougue
*   Bleu




In [ ]:
ground_truth = '''"### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions. Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading to a situation where they are dependent on the employer for their livelihood. This dependence can result in further suppression of wages and a decline in working conditions. Overall, the concept of monopsony is essential to understanding the dynamics of labor markets and the impact of market power on workers. Further research is needed to understand the extent and impact of monopsonies on the economy and to develop policies to address this issue. References: Bivens, J., & Mishel, L. (2013). The Pay of Corporate Executives and Financial Professionals as Evidence of Rents in Top 1 Percent Incomes. Journal of Economic Perspectives, 27(3), 57-78.### Human: Now explain it to a dog'''
prediction = tokenizer.batch_decode(outputs)[0]



In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(prediction,ground_truth)
print(scores)


{'rouge1': Score(precision=0.36259541984732824, recall=0.39094650205761317, fmeasure=0.37623762376237624), 'rougeL': Score(precision=0.3015267175572519, recall=0.32510288065843623, fmeasure=0.3128712871287128)}


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
print('BLEU score -> {}'.format(sentence_bleu(prediction.split(), ground_truth.split() )))


BLEU score -> 4.523966563792634e-232


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
print('BLEU score -> {}'.format(sentence_bleu(prediction, ground_truth )))

BLEU score -> 6.916324452906089e-232
